In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as graph
import seaborn as sns

import cartopy.crs as ccrs

import statsmodels

import os
from tqdm import tqdm, trange

from convertbng.util import convert_bng, convert_lonlat
import utm

In [16]:
location_columns = ['waterbody_name', 'site_description', 'primary_bassin', 'secondary_bassin']

In [2]:
# code and species dictionary

code_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/Banff_NP_Freshwater_Lake_Fish_Index_2017_data_dictionary.csv')
# code_df = code_df.iloc[:-6]
code_df.rename(columns={'Data_Value_Valeur_de_la_donnée': 'code_name', 
                        'Value_Description_EN_Description_de_la_valeur': 'common_name'}, inplace=True)
code_df = code_df[['code_name', 'common_name']]
code_df['common_name'] = code_df['common_name'].str.lower().str.replace(' ', '_')
display(code_df.sample(5))

dict_code_name = code_df.set_index('code_name').to_dict()['common_name']

,code_name,common_name
34,LNDC,longnose_dace
41,SLSC,slimy_sculpin
33,LKWH,lake_whitefish
24,BURB,burbot
47,WALL,walleye


In [3]:
new_codes = {'LNDC': 'longnose_dace', 'BNTR': 'brown_trout', 'RNTR1': 'rainbow_trout', 'CTTR1': 'cutthroat_trout', 
             'WHSC': 'white_sucker', 'BLTRCTTR(N&I)': 'bull_trout_x_cutthroat_trout', 'CTTR(hybrids)': 'cutthroat_trout', 
             'CTTRhybrids': 'cutthroat_trout', 'SUCK': 'white_sucker', 'GRCR': 'grass_carp'}

dict_code_name.update(new_codes)

In [62]:
# Species dictionay

species_name_df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/species_name_dictionary.csv')

dict_species_name = species_name_df.set_index('common_name').to_dict()['scientific_name']

In [4]:
new_names = {'nine-spine_stickleback': 'pungitius_pungitius'}

dict_species_name.update(new_names)
dict_species_name

{'lake_sturgeon': 'acipenser_fulvescens',
 'green_sturgeon': 'acipenser_medirostris',
 'atlantic_sturgeon': 'acipenser_oxyrinchus',
 'white_sturgeon': 'acipenser_transmontanus',
 'sturgeon': 'acipenseridae_spp',
 'sturgeons': 'acipenseridae_spp',
 'chiselmouth': 'acrocheilus_alutaceus',
 'poachers': 'agonidae_spp',
 'poacher_spp': 'agonidae_spp',
 'northern_spearnose_poacher': 'agonopsis_vulsa',
 'northern_spparnose_poacher': 'agonopsis_vulsa',
 'blueback_herring': 'alosa_aestivalis',
 'skipjack_herring': 'alosa_chrysochloris',
 'alewife': 'alosa_pseudoharengus',
 'alewife_(gaspereau)': 'alosa_pseudoharengus',
 'american_shad': 'alosa_sapidissima',
 'allis_shad': 'alosa_spp',
 'rock_bass': 'ambloplites_rupestris',
 'tiger_salamander': 'ambystoma_tigrinum',
 'black_bullhead': 'ameiurus_melas',
 'black_catfish': 'ameiurus_melas',
 'bullhead': 'ameiurus_melas',
 'black_bullhead_x_brown_bullhead': 'ameiurus_melas_x_ameiurus_nebulosus',
 'yellow_bullhead': 'ameiurus_natalis',
 'brown_bullhe

In [6]:
species_name_df = pd.DataFrame.from_dict({'common_name': list(dict_species_name.keys()), 
                                        'scientific_name' : list(dict_species_name.values())})
# species_name_df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/species_name_dictionary.csv', index=False)

# Alberta

In [19]:
path = '/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/Stocking/'

stocking_df = []
length_df = []

for file in os.listdir(path):
    year = file.split('esrd')[0]
    print(year)
    
    df = pd.read_csv(f"{path}{file}")
    print(f"# species = {df['SPECIES'].unique()}")
    stocking_df.append(df)
    length_df.append(len(df))

print(length_df)

2009
# species = ['BKTR' 'RNTR' 'BNTR' 'WALL' 'ARGR' 'GRCR' 'CTTR' 'BLTR']
2010
# species = ['BNTR' 'BKTR' 'RNTR' 'WALL' 'CTTR' 'BLTR']
2011
# species = ['RNTR' 'BKTR' 'BNTR' 'WALL' 'GRCR' 'CTTR']
2012
# species = ['BKTR' 'RNTR' 'BNTR' 'ARGR' 'WALL' 'CTTR']
2013
# species = ['RNTR' 'BKTR' 'BNTR' 'ARGR' 'CTTR']
2014
# species = ['RNTR' 'BNTR' 'BKTR' 'CTTR' 'WALL']
2015
# species = ['RNTR' 'CTTR' 'BKTR' 'BNTR']
[539, 539, 570, 567, 455, 336, 79]


In [20]:
display(length_df)

[539, 539, 570, 567, 455, 336, 79]

In [21]:
stocking_df = pd.concat(stocking_df)
stocking_df = stocking_df[['LAT', 'LONG', 'SHIPMENT DATE', 'SPECIES', 'COMMON NAME', 'OFFICIAL NAME']]
print(len(stocking_df))

stocking_df['year'] = pd.DatetimeIndex(pd.to_datetime(stocking_df['SHIPMENT DATE'])).year

stocking_df['lat_long'] = stocking_df['LAT'].round(4).astype(str) + '_' + stocking_df['LONG'].round(4).astype(str)
stocking_df.rename(columns = {'LAT': 'latitude', 'LONG': 'longitude', 'OFFICIAL NAME': 'waterbody_name', 
                              'COMMON NAME': 'site_description'}, inplace=True)

display(stocking_df.head())

3085


/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,latitude,longitude,SHIPMENT DATE,SPECIES,site_description,waterbody_name,year,lat_long
0,49.671905,-112.879725,4/17/2009,BKTR,NICHOLAS SHERAN PARK POND,UNNAMED,2009,49.6719_-112.8797
1,49.407453,-112.869274,4/18/2009,RNTR,MAGRATH CHILDREN'S POND,UNNAMED,2009,49.4075_-112.8693
2,49.647070,-112.459320,4/18/2009,RNTR,MCQUILLAN RESERVOIR,UNNAMED,2009,49.6471_-112.4593
3,49.095497,-111.995479,4/18/2009,RNTR,GOLDSPRING PARK POND,UNNAMED,2009,49.0955_-111.9955
4,49.862522,-113.507613,4/18/2009,RNTR,GRANUM POND,UNNAMED,2009,49.8625_-113.5076


In [22]:
for col in tqdm(stocking_df['SPECIES'].unique()):
    stocking_df[col] = np.where(stocking_df.isin([col]).any(1), 1, np.nan)
    stocking_df[col] = stocking_df[col].fillna(stocking_df.groupby(['year', 'lat_long'])[col].transform('mean'))
print(stocking_df['year'].min(), stocking_df['year'].max())

display(stocking_df.sample(5))

100%|██████████| 8/8 [00:00<00:00, 100.47it/s]

2009 2015


,latitude,longitude,SHIPMENT DATE,SPECIES,site_description,waterbody_name,year,lat_long,BKTR,RNTR,BNTR,WALL,ARGR,GRCR,CTTR,BLTR
102,53.517861,-114.138808,5/2/2012,RNTR,COTTAGE,SPRING LAKE,2012,53.5179_-114.1388,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
165,55.021444,-112.015324,5/11/2010,RNTR,MILE 07 LAKE,UNNAMED,2010,55.0214_-112.0153,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
486,52.451617,-116.149976,9/11/2012,RNTR,FISH LAKE,SHUNDA LAKE,2012,52.4516_-116.15,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
309,52.413889,-115.001806,5/12/2014,RNTR,,TWIN LAKES,2014,52.4139_-115.0018,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
302,52.011001,-114.309334,5/30/2013,RNTR,DICKSON TROUT POND,UNNAMED,2013,52.011_-114.3093,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
stocking_df.rename(columns=dict_code_name, inplace=True)
stocking_df.rename(columns=dict_species_name, inplace=True)
display(stocking_df.head())

,latitude,longitude,SHIPMENT DATE,SPECIES,site_description,waterbody_name,year,lat_long,salvelinus_fontinalis,oncorhynchus_mykiss,salmo_trutta,sander_vitreus,thymallus_arcticus,ctenopharyngodon_idella,oncorhynchus_clarkii,salvelinus_confluentus
0,49.671905,-112.879725,4/17/2009,BKTR,NICHOLAS SHERAN PARK POND,UNNAMED,2009,49.6719_-112.8797,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49.407453,-112.869274,4/18/2009,RNTR,MAGRATH CHILDREN'S POND,UNNAMED,2009,49.4075_-112.8693,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,49.647070,-112.459320,4/18/2009,RNTR,MCQUILLAN RESERVOIR,UNNAMED,2009,49.6471_-112.4593,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,49.095497,-111.995479,4/18/2009,RNTR,GOLDSPRING PARK POND,UNNAMED,2009,49.0955_-111.9955,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,49.862522,-113.507613,4/18/2009,RNTR,GRANUM POND,UNNAMED,2009,49.8625_-113.5076,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(len(stocking_df))
stocking_df.drop(columns=['SHIPMENT DATE', 'SPECIES'], inplace=True)
stocking_df.drop_duplicates(inplace=True)
print(len(stocking_df))

display(stocking_df.head())
stocking_df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/stocking_2009_2015.csv', index=False)

3085
1618


,latitude,longitude,site_description,waterbody_name,year,lat_long,salvelinus_fontinalis,oncorhynchus_mykiss,salmo_trutta,sander_vitreus,thymallus_arcticus,ctenopharyngodon_idella,oncorhynchus_clarkii,salvelinus_confluentus
0,49.671905,-112.879725,NICHOLAS SHERAN PARK POND,UNNAMED,2009,49.6719_-112.8797,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49.407453,-112.869274,MAGRATH CHILDREN'S POND,UNNAMED,2009,49.4075_-112.8693,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,49.647070,-112.459320,MCQUILLAN RESERVOIR,UNNAMED,2009,49.6471_-112.4593,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,49.095497,-111.995479,GOLDSPRING PARK POND,UNNAMED,2009,49.0955_-111.9955,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,49.862522,-113.507613,GRANUM POND,UNNAMED,2009,49.8625_-113.5076,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


### Fish Survey

In [33]:
# Fish survey

df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/FishSurvey.csv')
df = df[['SPEC_CODE', 'SPEC_NAME', 'TERMINATIO', 'SURVEY_TYP', 'WTB_OFFICI',
         'LATITUDE', 'LONGITUDE']]
print(len(df))
display(df.head())

393469


,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,WTB_OFFICI,LATITUDE,LONGITUDE
0,NRPK,NORTHERN PIKE,8/11/1981 0:00:00,Seine;Trawl,ADAIR CREEK,59.077800,-117.747883
1,LKCH,LAKE CHUB,10/7/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405
2,LKCH,LAKE CHUB,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405
3,LKCH,LAKE CHUB,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.215301,-113.187304
4,LKCH,LAKE CHUB,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405


In [34]:
df = df[df['TERMINATIO'] != 'Test Net']

df['year'] = pd.DatetimeIndex(pd.to_datetime(df['TERMINATIO'])).year

df['lat_long'] = df['LATITUDE'].round(3).astype(str) + '_' + df['LONGITUDE'].round(3).astype(str)
df['SPEC_NAME'] = df['SPEC_NAME'].str.lower().str.replace(' ', '_')

df.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df.dropna(subset=['SPEC_CODE'], inplace=True)
df.rename(columns = {'WTB_OFFICI': 'waterbody_name', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'}, inplace=True)
print(len(df))

display(df.tail())

335952


,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,waterbody_name,latitude,longitude,year,lat_long
393457,WHSC,white_sucker,6/19/2013 0:00:00,"Trap nets, Minnow Traps, Dip Nets",ZAMA RIVER,59.045663,-118.901958,2013.0,59.046_-118.902
393458,WHSC,white_sucker,7/16/2014 0:00:00,"Trap nets, Minnow Traps, Dip Nets",ZAMA RIVER,59.045635,-118.902048,2014.0,59.046_-118.902
393459,BRST,brook_stickleback,7/12/2014 0:00:00,"Trap nets, Minnow Traps, Dip Nets",UNNAMED,59.040185,-118.906105,2014.0,59.04_-118.906
393461,FNDC,finescale_dace,7/16/2014 0:00:00,"Trap nets, Minnow Traps, Dip Nets",ZAMA RIVER,59.045635,-118.902048,2014.0,59.046_-118.902
393462,BRST,brook_stickleback,7/16/2014 0:00:00,"Trap nets, Minnow Traps, Dip Nets",ZAMA RIVER,59.045635,-118.902048,2014.0,59.046_-118.902


In [117]:
# df[df['SURVEY_COM'].notna()]['SURVEY_COM'].to_csv('/mnt/c/Users/imrit/Desktop/survey_comments.csv', index=False)

/home/arshad/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [35]:
%%time
df['SPEC_NAME'].replace(dict_species_name, inplace=True)

CPU times: user 7.11 s, sys: 78.1 ms, total: 7.19 s
Wall time: 7.28 s


In [36]:
display(df.head())

,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,waterbody_name,latitude,longitude,year,lat_long
0,NRPK,esox_lucius,8/11/1981 0:00:00,Seine;Trawl,ADAIR CREEK,59.077800,-117.747883,1981.0,59.078_-117.748
1,LKCH,couesius_plumbeus,10/7/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2
2,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2
3,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.215301,-113.187304,2015.0,58.215_-113.187
4,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2


In [37]:
%%time
for col in tqdm(df['SPEC_NAME'].unique()):
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))
    
display(df.head())

100%|██████████| 82/82 [04:44<00:00,  3.47s/it]


,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,waterbody_name,latitude,longitude,year,lat_long,esox_lucius,...,hybognathus_argyritis,cottus_spp,noturus_flavus,gasterosteus_aculeatus,triops,cichlidae_spp,cyprinus_rubrofuscus,coregonus_zenithicus,micropterus_dolomieu,ctenopharyngodon_idella
0,NRPK,esox_lucius,8/11/1981 0:00:00,Seine;Trawl,ADAIR CREEK,59.077800,-117.747883,1981.0,59.078_-117.748,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LKCH,couesius_plumbeus,10/7/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.215301,-113.187304,2015.0,58.215_-113.187,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LKCH,couesius_plumbeus,10/2/2015 0:00:00,Electrofishing,ALICE CREEK,58.288301,-113.200405,2015.0,58.288_-113.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CPU times: user 2min 27s, sys: 2min 15s, total: 4min 43s
Wall time: 4min 44s


In [38]:
print(df['year'].min(), df['year'].max())

1942.0 2020.0


In [43]:
df.drop(columns=['SPEC_CODE', 'SPEC_NAME', 'TERMINATIO', 'SURVEY_TYP'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
display(df.sample(5))

df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/fish_survey_occurence_1942_2020.csv', 
          index=False)

335952
63837


,waterbody_name,latitude,longitude,year,lat_long,esox_lucius,couesius_plumbeus,culaea_inconstans,catostomus_commersonii,chrosomus_neogaeus,...,hybognathus_argyritis,cottus_spp,noturus_flavus,gasterosteus_aculeatus,triops,cichlidae_spp,cyprinus_rubrofuscus,coregonus_zenithicus,micropterus_dolomieu,ctenopharyngodon_idella
138084,LESSER SLAVE LAKE,55.509979,-114.933044,1999.0,55.51_-114.933,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250541,OTTER RIVER,56.726657,-116.247678,2011.0,56.727_-116.248,NaN,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346719,VICARY CREEK,49.755272,-114.466179,2004.0,49.755_-114.466,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37725,REDWILLOW RIVER,55.050096,-119.355977,2009.0,55.05_-119.356,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73425,NARROW LAKE,54.622840,-113.612121,2018.0,54.623_-113.612,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Fish Culture Stocking

In [46]:
# Fish culture stocking

df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/FishCultureStocking.csv')
df.head()

,OID,SPEC_CODE,SPEC_NAME,INV_PROJ_I,TERMINATIO,LATITUDE,LONGITUDE,UTM_NORTHI,UTM_EASTIN,ATS_RANGE,...,UTM_MERIDI,WTB_ID,TTM_NORTHI,TTM_EASTIN,GENOTYPE,STRAIN,AVG_WEIGHT,STATION,AVG_LENGTH,Stocking_D
0,-1,NRPK,NORTHERN PIKE,18606,8/15/1957 0:00:00,54.053198,-111.372081,5989504.554,475641.42,10,...,-111 (Zone 12),5509,5993130.886,737360.817,2N,Historical Entry - Strain Unknown,1500,Unknown Station,0,
1,-1,WALL,WALLEYE,18609,7/2/1994 0:00:00,54.143062,-111.511662,5999559.958,466576.03,11,...,-111 (Zone 12),6063,6002660.041,727738.931,2N,Bistcho Lake,0.39,Cold Lake,3.73,
2,-1,WALL,WALLEYE,18608,7/20/1989 0:00:00,54.61404,-112.726508,6053213.249,388505.76,18,...,-111 (Zone 12),5820,6051796.088,646753.198,2N,Historical Entry - Strain Unknown,0.95,Cold Lake,5.02,
3,-1,NRPK,NORTHERN PIKE,18606,5/15/1968 0:00:00,54.176603,-111.51632,6003294.01,466299.015,11,...,-111 (Zone 12),5181,6006373.258,727250.959,2N,Historical Entry - Strain Unknown,0.01,Unknown Station,0,
4,-1,LKWH,LAKE WHITEFISH,18589,4/11/1932 0:00:00,54.362184,-111.889992,6024184.689,442169.748,13,...,-111 (Zone 12),6149,6025865.938,701973.256,2N,Historical Entry - Strain Unknown,0.01,Canyon Creek Hatchery,0,


In [47]:
df = df[df['TERMINATIO'] != 'TERMINATIO']
df['year'] = pd.DatetimeIndex(pd.to_datetime(df['TERMINATIO'])).year
df = df[['SPEC_CODE', 'SPEC_NAME', 'TERMINATIO', 'SURVEY_TYP', 'LATITUDE', 'LONGITUDE', 'WTB_OFFICI', 'STATION', 'year']]

for col in ['LATITUDE', 'LONGITUDE']:
    df[col] = df[col].astype(float)
    
df['lat_long'] = df['LATITUDE'].round(3).astype(str) + '_' + df['LONGITUDE'].round(3).astype(str)
df['SPEC_NAME'] = df['SPEC_NAME'].str.lower().str.replace(' ', '_')
df.rename(columns = {'WTB_OFFICI': 'waterbody_name', 'LATITUDE': 'latitude', 'LONGITUDE': 'longitude', 
                     'STATION': 'site_description'}, inplace=True)

df.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df.dropna(subset=['SPEC_CODE'], inplace=True)
print(len(df))

display(df.tail())

21745


,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,latitude,longitude,waterbody_name,site_description,year,lat_long
21742,RNTR,rainbow_trout,5/14/2003 0:00:00,Fish Culture Stocking,59.161004,-118.681101,UNNAMED,Cold Lake,2003,59.161_-118.681
21743,RNTR,rainbow_trout,6/15/1991 0:00:00,Fish Culture Stocking,59.161004,-118.681101,UNNAMED,Private Trout Producer,1991,59.161_-118.681
21744,RNTR,rainbow_trout,5/23/2007 0:00:00,Fish Culture Stocking,59.161004,-118.681101,UNNAMED,Cold Lake,2007,59.161_-118.681
21745,RNTR,rainbow_trout,5/30/1995 0:00:00,Fish Culture Stocking,59.161004,-118.681101,UNNAMED,Cold Lake,1995,59.161_-118.681
21746,RNTR,rainbow_trout,5/22/2013 0:00:00,Fish Culture Stocking,59.161004,-118.681101,UNNAMED,Cold Lake,2013,59.161_-118.681


In [48]:
%%time
df.replace(dict_species_name, inplace=True)

CPU times: user 2.28 s, sys: 0 ns, total: 2.28 s
Wall time: 2.29 s


In [49]:
for col in tqdm(df['SPEC_NAME'].unique()):
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('mean'))

print(df['year'].min(), df['year'].max())
display(df.sample(5))

100%|██████████| 23/23 [00:00<00:00, 29.97it/s]

1922 2020


,SPEC_CODE,SPEC_NAME,TERMINATIO,SURVEY_TYP,latitude,longitude,waterbody_name,site_description,year,lat_long,...,oncorhynchus_aguabonita,salvelinus_namaycush,oncorhynchus_kisutch,salvelinus_confluentus,percopsis_omiscomaycus,pimephales_promelas,culaea_inconstans,coregonus_artedi,salvelinus_fontinalis_x_salvelinus_namaycush,micropterus_dolomieu
18064,RNTR,oncorhynchus_mykiss,5/15/1961 0:00:00,Fish Culture Stocking,51.667718,-113.236860,BRACONNIER RESERVOIR,Unknown Station,1961,51.668_-113.237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14886,WALL,sander_vitreus,7/10/2000 0:00:00,Fish Culture Stocking,52.374063,-114.187701,SYLVAN LAKE,Cold Lake,2000,52.374_-114.188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16121,BKTR,salvelinus_fontinalis,5/31/2012 0:00:00,Fish Culture Stocking,54.786102,-115.194645,CHRYSTINA LAKE,Cold Lake,2012,54.786_-115.195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15186,RNTR,oncorhynchus_mykiss,5/15/1980 0:00:00,Fish Culture Stocking,51.208304,-112.955705,UNNAMED,Unknown Station,1980,51.208_-112.956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9854,RNTR,oncorhynchus_mykiss,5/27/2005 0:00:00,Fish Culture Stocking,50.035398,-112.816726,UNNAMED,Private Trout Producer,2005,50.035_-112.817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.drop(columns=['SPEC_CODE', 'SPEC_NAME', 'SURVEY_TYP', 'TERMINATIO'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/fish_culture_stocking_occurence_1922_2020.csv', 
          index=False)

21745
15639


,latitude,longitude,waterbody_name,site_description,year,lat_long,esox_lucius,sander_vitreus,coregonus_clupeaformis,notropis_hudsonius,...,oncorhynchus_aguabonita,salvelinus_namaycush,oncorhynchus_kisutch,salvelinus_confluentus,percopsis_omiscomaycus,pimephales_promelas,culaea_inconstans,coregonus_artedi,salvelinus_fontinalis_x_salvelinus_namaycush,micropterus_dolomieu
12351,53.376492,-114.309875,UNNAMED,Cold Lake,2011,53.376_-114.31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3630,52.254167,-114.850000,CLEARWATER RIVER,Banff Hatchery,1936,52.254_-114.85,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4450,49.655563,-114.559795,UNNAMED,Sam Livingston,1996,49.656_-114.56,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15028,52.740747,-110.609119,UNNAMED,Sam Livingston,1991,52.741_-110.609,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17311,53.618628,-114.074339,CHICKAKOO LAKE,Cold Lake,2010,53.619_-114.074,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Waterton

In [63]:
# Waterton

df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/Waterton_Lakes_NP_Freshwater_Stream_Fish_Occupancy_2016_data.csv')
df.columns = df.columns.str.replace(' ', '_')

df['latitude'], df['longitude'] = np.where(df['Universal_Transverse_Mercator_Zone'] == 12, 
                                           (utm.to_latlon(df['Easting_coordinate'], df['Northing_coordinate'], 12, 'U')), 
                                           np.nan)
df['latitude'], df['longitude'] = np.where(df['Universal_Transverse_Mercator_Zone'] == 11, 
                                           (utm.to_latlon(df['Easting_coordinate'], df['Northing_coordinate'], 11, 'U')), 
                                           (df['latitude'], df['longitude']))

display(df.head())

,Watershed,Site,Stream,Universal_Transverse_Mercator_Zone,Easting_coordinate,Northing_coordinate,latitude,longitude,Stream_Order,Wetted_Width_(meters),...,brook_trout,bull_trout,rainbow_trout,longnose_dace,longnose_sucker,white_sucker,trout_perch,burbot,CTTR/RBTR_hybrid,brown_trout
0,WATERTON,18,Crooked,11,284838,5445388,49.123630,-119.949173,4,5.3,...,0.0,0.0,0.0,29.0,12.0,16.0,14.0,1.0,NaN,NaN
1,WATERTON,18,Crooked,11,284838,5445388,49.123630,-119.949173,4,5.3,...,0.0,0.0,0.0,13.0,NaN,1.0,7.0,NaN,NaN,NaN
2,WATERTON,21,Blakiston,11,716887,5445698,49.125808,-114.027050,3,3.9,...,0.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WATERTON,21,Blakiston,11,716887,5445698,49.125808,-114.027050,3,3.9,...,0.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WATERTON,24,Blue Grouse,11,709176,5444564,49.118292,-114.133194,1,1.9,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df.rename(columns=dict_species_name, inplace=True)
# df_species_names = [x for x in list(dict_species_name.values()) if x in df.columns]
# df_species_names = list(dict.fromkeys(df_species_names))

# waterton_cols = ['latitude', 'longitude', 'year']
# waterton_cols.extend(df_species_names)

df.replace('No Rep', '99/99/2016', inplace=True)
df['year'] = df['DATE_(date/month/year)'].str.split('/').str.get(2).astype(int)

df.rename(columns = {'Watershed': 'primary_bassin', 'Stream': 'waterbody_name'}, inplace=True)
df['site_decription'] = 'stream'

df.drop(columns=['Site', 'Universal_Transverse_Mercator_Zone', 'Easting_coordinate', 'Northing_coordinate', 'Stream_Order', 
                 'Wetted_Width_(meters)', 'Bankful_Width_(meters)', 'Notes', 'Repetition', 'DATE_(date/month/year)'], 
        inplace=True)

In [65]:
df['lat_long'] = df['latitude'].round(3).astype(str) + '_' + df['longitude'].round(3).astype(str)
df.head()

,primary_bassin,waterbody_name,latitude,longitude,oncorhynchus_clarkii,salvelinus_fontinalis,salvelinus_confluentus,oncorhynchus_mykiss,rhinichthys_cataractae,catostomus_catostomus,catostomus_commersonii,percopsis_omiscomaycus,lota_lota,oncorhynchus_clarkii_x_oncorhynchus_mykiss,salmo_trutta,year,site_decription,lat_long
0,WATERTON,Crooked,49.123630,-119.949173,0.0,0.0,0.0,0.0,29.0,12.0,16.0,14.0,1.0,NaN,NaN,2016,stream,49.124_-119.949
1,WATERTON,Crooked,49.123630,-119.949173,0.0,0.0,0.0,0.0,13.0,NaN,1.0,7.0,NaN,NaN,NaN,2016,stream,49.124_-119.949
2,WATERTON,Blakiston,49.125808,-114.027050,0.0,0.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.126_-114.027
3,WATERTON,Blakiston,49.125808,-114.027050,0.0,0.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.126_-114.027
4,WATERTON,Blue Grouse,49.118292,-114.133194,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.118_-114.133


In [71]:
for col in tqdm(df.columns[4:-3]):
    df[col] = np.where(df[col].notnull(), 1, df[col])
    df[col] = df[col].fillna(df.groupby(['year', 'lat_long'])[col].transform('sum'))

df.replace(0, np.nan, inplace=True)
    
display(df.head())

100%|██████████| 11/11 [00:00<00:00, 227.59it/s]


,primary_bassin,waterbody_name,latitude,longitude,oncorhynchus_clarkii,salvelinus_fontinalis,salvelinus_confluentus,oncorhynchus_mykiss,rhinichthys_cataractae,catostomus_catostomus,catostomus_commersonii,percopsis_omiscomaycus,lota_lota,oncorhynchus_clarkii_x_oncorhynchus_mykiss,salmo_trutta,year,site_decription,lat_long
0,WATERTON,Crooked,49.123630,-119.949173,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,2016,stream,49.124_-119.949
1,WATERTON,Crooked,49.123630,-119.949173,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,2016,stream,49.124_-119.949
2,WATERTON,Blakiston,49.125808,-114.027050,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.126_-114.027
3,WATERTON,Blakiston,49.125808,-114.027050,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.126_-114.027
4,WATERTON,Blue Grouse,49.118292,-114.133194,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.118_-114.133


In [72]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
display(df.head())

df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/waterton_occurrence_2016_1.csv', index=False)

54
27


,primary_bassin,waterbody_name,latitude,longitude,oncorhynchus_clarkii,salvelinus_fontinalis,salvelinus_confluentus,oncorhynchus_mykiss,rhinichthys_cataractae,catostomus_catostomus,catostomus_commersonii,percopsis_omiscomaycus,lota_lota,oncorhynchus_clarkii_x_oncorhynchus_mykiss,salmo_trutta,year,site_decription,lat_long
0,WATERTON,Crooked,49.123630,-119.949173,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,2016,stream,49.124_-119.949
2,WATERTON,Blakiston,49.125808,-114.027050,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.126_-114.027
4,WATERTON,Blue Grouse,49.118292,-114.133194,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.118_-114.133
6,WATERTON,Blakiston,49.112009,-114.078900,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.112_-114.079
8,WATERTON,Bauerman,49.151866,-114.085320,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,stream,49.152_-114.085


In [75]:
# Waterton 2011

df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Used/Waterton_Lakes_NP_Freshwater_Lake_Fish_Index_2016_data.csv')
df = df[['Lake Name', 'Area (ha)', 'Species - Present (2011)']]
df.rename(columns={'Species - Present (2011)': 'species_present', 'Lake Name': 'waterbody_name'}, inplace=True)
df.dropna(inplace=True)

df.replace('cuttthroat_trout', 'cutthroat_trout', inplace=True)

df.replace(dict_species_name, inplace=True)
print(len(df))
display(df.head())

77


,waterbody_name,Area (ha),species_present
0,Lineham North,18.96,oncorhynchus_clarkii
1,Lineham North,18.96,oncorhynchus_mykiss
4,Lineham Hourglass,12.64,oncorhynchus_clarkii
9,Cameron,172.30,salvelinus_fontinalis
10,Cameron,172.30,oncorhynchus_mykiss


In [76]:
df['latitude'] = 'TO_GET'
df['longitude'] = 'TO_GET'
df['year'] = 2011

for col in df['species_present'].unique():
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby('waterbody_name')[col].transform('mean'))
    
display(df.sample(10))

,waterbody_name,Area (ha),species_present,latitude,longitude,year,oncorhynchus_clarkii,oncorhynchus_mykiss,salvelinus_fontinalis,salvelinus_namaycush,...,coregonus_clupeaformis,prosopium_coulterii,esox_lucius,lota_lota,myoxocephalus_thompsonii,couesius_plumbeus,catostomus_catostomus,catostomus_commersonii,percopsis_omiscomaycus,salmo_trutta
52,Lower (Middle) Waterton,429.00,lota_lota,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
85,Maskinonge,65.00,catostomus_catostomus,TO_GET,TO_GET,2011,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
54,Lower (Middle) Waterton,429.00,couesius_plumbeus,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
9,Cameron,172.30,salvelinus_fontinalis,TO_GET,TO_GET,2011,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,Upper Waterton,941.00,couesius_plumbeus,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50,Lower (Middle) Waterton,429.00,coregonus_clupeaformis,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
104,Park Entrance Pond,1.00,oncorhynchus_clarkii,TO_GET,TO_GET,2011,1.0,1.0,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,Lower (Middle) Waterton,429.00,catostomus_catostomus,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
44,Linnet,3.52,catostomus_catostomus,TO_GET,TO_GET,2011,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
48,Lower (Middle) Waterton,429.00,salvelinus_namaycush,TO_GET,TO_GET,2011,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0


In [77]:
print(len(df))
df.drop(columns=['species_present'], inplace=True)
df.drop_duplicates(inplace=True)
print(len(df))
display(df.head())

77
24


,waterbody_name,Area (ha),latitude,longitude,year,oncorhynchus_clarkii,oncorhynchus_mykiss,salvelinus_fontinalis,salvelinus_namaycush,salvelinus_confluentus,...,coregonus_clupeaformis,prosopium_coulterii,esox_lucius,lota_lota,myoxocephalus_thompsonii,couesius_plumbeus,catostomus_catostomus,catostomus_commersonii,percopsis_omiscomaycus,salmo_trutta
0,Lineham North,18.96,TO_GET,TO_GET,2011,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lineham Hourglass,12.64,TO_GET,TO_GET,2011,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Cameron,172.30,TO_GET,TO_GET,2011,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Akamina,4.65,TO_GET,TO_GET,2011,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Lower Carthiew,7.33,TO_GET,TO_GET,2011,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/waterton_2011_occurence_NO_LAT_LONG.csv', 
          index=True)

### Banff

In [79]:
# Banff

df = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Used/Banff_NP_Freshwater_Lake_Fish_Index_2017_data.csv')
df = df[['Lake Name', 'Present Condition']]
df.rename(columns={'Present Condition': 'present_condition', 'Lake Name': 'waterbody_name'}, inplace=True)
df = df[~df['present_condition'].isin(['Fishless ', 'Fishless'])]
df['year'] = 2017
df['latitude'], df['longitude'] = 'TO_GET', 'TO_GET'
display(df)

,waterbody_name,present_condition,year,latitude,longitude
0,Alexandra Pond,SPLK,2017,TO_GET,TO_GET
4,Altrude (2),WSCT,2017,TO_GET,TO_GET
7,Arnica,WSCT,2017,TO_GET,TO_GET
8,Badger,BKTR,2017,TO_GET,TO_GET
9,Baker Lake,BKTR,2017,TO_GET,TO_GET
...,...,...,...,...,...
164,Warden Lake - upper,"BKTR, RNTR, SPLK",2017,TO_GET,TO_GET
165,Watchman Lake,CTTR,2017,TO_GET,TO_GET
166,Waterfowl (Lower),"BKTR, RNTR",2017,TO_GET,TO_GET
167,Waterfowl (Upper),"BKTR, RNTR",2017,TO_GET,TO_GET


In [80]:
df = df.assign(present_condition=df['present_condition'].str.split(',')).explode('present_condition')
df['present_condition'] = df['present_condition'].str.replace(' ', '')
display(df)

,waterbody_name,present_condition,year,latitude,longitude
0,Alexandra Pond,SPLK,2017,TO_GET,TO_GET
4,Altrude (2),WSCT,2017,TO_GET,TO_GET
7,Arnica,WSCT,2017,TO_GET,TO_GET
8,Badger,BKTR,2017,TO_GET,TO_GET
9,Baker Lake,BKTR,2017,TO_GET,TO_GET
...,...,...,...,...,...
166,Waterfowl (Lower),BKTR,2017,TO_GET,TO_GET
166,Waterfowl (Lower),RNTR,2017,TO_GET,TO_GET
167,Waterfowl (Upper),BKTR,2017,TO_GET,TO_GET
167,Waterfowl (Upper),RNTR,2017,TO_GET,TO_GET


In [81]:
df.replace(dict_code_name, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df.replace(dict_species_name, inplace=True)
df.head()

,waterbody_name,present_condition,year,latitude,longitude
0,Alexandra Pond,salvelinus_fontinalis_x_salvelinus_namaycush,2017,TO_GET,TO_GET
1,Altrude (2),oncorhynchus_clarkii,2017,TO_GET,TO_GET
2,Arnica,oncorhynchus_clarkii,2017,TO_GET,TO_GET
3,Badger,salvelinus_fontinalis,2017,TO_GET,TO_GET
4,Baker Lake,salvelinus_fontinalis,2017,TO_GET,TO_GET


In [83]:
for col in tqdm(df['present_condition'].unique()):
    df[col] = np.where(df.isin([col]).any(1), 1, np.nan)
    df[col] = df[col].fillna(df.groupby('waterbody_name')[col].transform('mean'))

display(df.head())

100%|██████████| 15/15 [00:00<00:00, 256.33it/s]


,waterbody_name,present_condition,year,latitude,longitude,salvelinus_fontinalis_x_salvelinus_namaycush,oncorhynchus_clarkii,salvelinus_fontinalis,salvelinus_alpinus,salvelinus_confluentus,salvelinus_namaycush,prosopium_williamsoni,oncorhynchus_mykiss,catostomus_catostomus,catostomus_commersonii,coregonus_artedi,salvelinus_confluentus_x_oncorhynchus_clarkii,salmo_trutta,coregonus_clupeaformis,rhinichthys_cataractae
0,Alexandra Pond,salvelinus_fontinalis_x_salvelinus_namaycush,2017,TO_GET,TO_GET,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Altrude (2),oncorhynchus_clarkii,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnica,oncorhynchus_clarkii,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Badger,salvelinus_fontinalis,2017,TO_GET,TO_GET,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Baker Lake,salvelinus_fontinalis,2017,TO_GET,TO_GET,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df.drop(columns=['present_condition'], inplace=True)
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

display(df.sample(5))

155
92


,waterbody_name,year,latitude,longitude,salvelinus_fontinalis_x_salvelinus_namaycush,oncorhynchus_clarkii,salvelinus_fontinalis,salvelinus_alpinus,salvelinus_confluentus,salvelinus_namaycush,prosopium_williamsoni,oncorhynchus_mykiss,catostomus_catostomus,catostomus_commersonii,coregonus_artedi,salvelinus_confluentus_x_oncorhynchus_clarkii,salmo_trutta,coregonus_clupeaformis,rhinichthys_cataractae
127,Turquoise Lake,2017,TO_GET,TO_GET,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnica,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Moose Lake,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Fish Lakes,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Bow Lake,2017,TO_GET,TO_GET,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/Banff_NP_freshwater_occurence_2017_NO_LAT_LONG.csv', 
          index=True)

### Wild Fish communities, Oil Sands Region


In [86]:
df_2013 = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/WildFishCommFishHealth-CommDePoissonsSauvagesSanteDesPoissons-2013-v1.csv')
df_2013.drop(columns=['Fork Length (mm) / La longueur à la fourche (mm)', 'Total Weight (g) / Poids total (g)', 
                      'Notes / Remarques', 'Site Name / Nom du site', 'Reach Code / Code de allonge'], inplace=True)

df_2013.head()

,Waterbody Name / Le nom du plan d'eau,Site & Description / Site et description,Latitude (Decimal Degrees) / Latitude (degrés décimaux),Longitude (Decimal Degrees) / Longitude (degrés décimaux),Date (yyyy-mm-dd) / Date (aaaa-mm-jj),Species / Espèce
0,Ells River,Upper Ells,57.229000,-111.890361,2013-09-23,Finescale Dace (FNDC)
1,Ells River,Upper Ells,57.229000,-111.890361,2013-09-23,Finescale Dace (FNDC)
2,Ells River,Upper Ells,57.229000,-111.890361,2013-09-23,Finescale Dace (FNDC)
3,Dunkirk River,Dunkirk - West of Fort McMurray,56.854766,-112.708400,2013-09-24,Longnose Dace (LNDC)
4,Dunkirk River,Dunkirk - West of Fort McMurray,56.854766,-112.708400,2013-09-24,Longnose Dace (LNDC)


In [87]:
wild_fish_com_col_names = {'Site Name / Nom du site': 'site_name', "Waterbody Name / Le nom du plan d'eau": 'waterbody_name', 
                           'Site & Description / Site et description': 'site_description', 
                           'Reach Code / Code de allonge': 'reach_code', 
                           'Latitude (Decimal Degrees) / Latitude (degrés décimaux)': 'latitude', 
                           'Longitude (Decimal Degrees) / Longitude (degrés décimaux)': 'longitude', 
                           'Date (yyyy-mm-dd) / Date (aaaa-mm-jj)': 'date', 'Species / Espèce': 'species'}

In [88]:
df_2013.rename(columns=wild_fish_com_col_names, inplace=True)
df_2013['year'] = pd.DatetimeIndex(pd.to_datetime(df_2013['date'])).year

print(len(df_2013))
df_2013.drop(columns=['date'], inplace=True)
df_2013.drop_duplicates(inplace=True)
print(len(df_2013))

df_2013['lat_long'] = df_2013['latitude'].round(3).astype(str) + '_' +  df_2013['longitude'].round(3).astype(str)

df_2013['species'] = df_2013['species'].str.split("\(").str.get(0).str.replace(' ', '_').str.rstrip('_').str.lower()
df_2013.replace(dict_species_name, inplace=True)
display(df_2013.sample(5))

954
25


,waterbody_name,site_description,latitude,longitude,species,year,lat_long
351,Steepbank River,Upper Steepbank,56.851905,-112.262351,rhinichthys_cataractae,2013,56.852_-112.262
912,Ells River,Upper Ells,57.229000,-111.890361,cottus_ricei,2013,57.229_-111.89
932,Ells River,Upper Ells,57.229000,-111.890361,catostomus_commersonii,2013,57.229_-111.89
105,Steepbank River,Lower Steepbank,57.023410,-111.473296,rhinichthys_cataractae,2013,57.023_-111.473
421,Ells River,Middle RAMP,57.221307,-111.988604,rhinichthys_cataractae,2013,57.221_-111.989


In [89]:
for col in tqdm(df_2013['species'].unique()):
    df_2013[col] = np.where(df_2013.isin([col]).any(1), 1, np.nan)
    df_2013[col] = df_2013[col].fillna(df_2013.groupby('lat_long')[col].transform('mean'))

display(df_2013.sample(5))

100%|██████████| 8/8 [00:00<00:00, 260.91it/s]


,waterbody_name,site_description,latitude,longitude,species,year,lat_long,chrosomus_neogaeus,rhinichthys_cataractae,catostomus_catostomus,margariscus_margarita,cottus_cognatus,cottus_ricei,percopsis_omiscomaycus,catostomus_commersonii
114,Firebag River,Middle Firebag,57.434495,-110.892203,rhinichthys_cataractae,2013,57.434_-110.892,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
911,Ells River,Middle RAMP,57.221307,-111.988604,cottus_ricei,2013,57.221_-111.989,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0
919,Ells River,Middle RAMP,57.221307,-111.988604,percopsis_omiscomaycus,2013,57.221_-111.989,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0
681,Horse River,Horse - West of Fort McMurray,56.361369,-112.176207,cottus_cognatus,2013,56.361_-112.176,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
20,High Hills River,High Hills - East of Fort McMurray,56.753900,-110.508300,rhinichthys_cataractae,2013,56.754_-110.508,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
print(len(df_2013))
df_2013.drop(columns=['species'], inplace=True)
df_2013.drop_duplicates(inplace=True)
print(len(df_2013))
display(df_2013)

25
10


,waterbody_name,site_description,latitude,longitude,year,lat_long,chrosomus_neogaeus,rhinichthys_cataractae,catostomus_catostomus,margariscus_margarita,cottus_cognatus,cottus_ricei,percopsis_omiscomaycus,catostomus_commersonii
0,Ells River,Upper Ells,57.229000,-111.890361,2013,57.229_-111.89,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0
3,Dunkirk River,Dunkirk - West of Fort McMurray,56.854766,-112.708400,2013,56.855_-112.708,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
20,High Hills River,High Hills - East of Fort McMurray,56.753900,-110.508300,2013,56.754_-110.508,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
26,Horse River,Horse - West of Fort McMurray,56.361369,-112.176207,2013,56.361_-112.176,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
51,Ells River,Lower Ells,57.266300,-111.734200,2013,57.266_-111.734,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN
105,Steepbank River,Lower Steepbank,57.023410,-111.473296,2013,57.023_-111.473,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
114,Firebag River,Middle Firebag,57.434495,-110.892203,2013,57.434_-110.892,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
125,Steepbank River,Middle Steepbank,56.991376,-111.339277,2013,56.991_-111.339,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
351,Steepbank River,Upper Steepbank,56.851905,-112.262351,2013,56.852_-112.262,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
421,Ells River,Middle RAMP,57.221307,-111.988604,2013,57.221_-111.989,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0


In [91]:
# 2017

df_2017 = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/WildFishCommFishHealth-CommDePoissionsSauvagesSanteDesPoissons-2017-v1.csv')
df_2017.drop(columns=['Fork Length (mm) / La longueur à la fourche (mm)', 'Total Weight (g) / Poids total (g)', 
                      'Notes / Remarques', 'Site Name / Nom du site', 'Reach Code / Code de allonge', 
                      'Total Length (mm) / Longueur total (mm)'], inplace=True)

df_2017.head()

,Waterbody Name / Le nom du plan d'eau,Site & Description / Site et description,Latitude (Decimal Degrees) / Latitude (degrés décimaux),Longitude (Decimal Degrees) / Longitude (degrés décimaux),Date (yyyy-mm-dd) / Date (aaaa-mm-jj),Species / Espèce
0,Christina River,Christina-2,56.7539,-110.5083,2017-10-03,Longnose Dace (Longnose Sucker (LNSC))
1,Christina River,Christina-2,56.7539,-110.5083,2017-10-03,Longnose Sucker (LNSC)
2,Christina River,Christina-2,56.7539,-110.5083,2017-10-03,Finescale Dace (FNDC)
3,Christina River,Christina-2,56.7539,-110.5083,2017-10-03,Finescale Dace (FNDC)
4,Christina River,Christina-2,56.7539,-110.5083,2017-10-03,Finescale Dace (FNDC)


In [92]:
df_2017.rename(columns=wild_fish_com_col_names, inplace=True)
df_2017['year'] = pd.DatetimeIndex(pd.to_datetime(df_2017['date'])).year

print(len(df_2017))
df_2017.drop(columns=['date'], inplace=True)
df_2017.drop_duplicates(inplace=True)
print(len(df_2017))

df_2017['lat_long'] = df_2017['latitude'].round(3).astype(str) + '_' +  df_2017['longitude'].round(3).astype(str)

df_2017['species'] = df_2017['species'].str.split("\(").str.get(0).str.replace(' ', '_').str.rstrip('_').str.lower()
df_2017.replace(dict_species_name, inplace=True)
display(df_2017.sample(5))

338
60


,waterbody_name,site_description,latitude,longitude,species,year,lat_long
203,Christina River,Christina-3,55.719685,-111.220095,esox_lucius,2017,55.72_-111.22
169,Christina River,Christina-4,55.888699,-111.539965,margariscus_margarita,2017,55.889_-111.54
62,Sunday Creek,Sunday-1,55.584120,-110.893442,esox_lucius,2017,55.584_-110.893
214,Christina River,Christina-3,55.719685,-111.220095,catostomus_catostomus,2017,55.72_-111.22
15,Christina River,Christina-2,56.753900,-110.508300,rhinichthys_cataractae,2017,56.754_-110.508


In [93]:
for col in df_2017['species'].unique():
    df_2017[col] = np.where(df_2017.isin([col]).any(1), 1, np.nan)
    df_2017[col] = df_2017[col].fillna(df_2017.groupby('lat_long')[col].transform('mean'))

display(df_2017.sample(5))

,waterbody_name,site_description,latitude,longitude,species,year,lat_long,rhinichthys_cataractae,catostomus_catostomus,chrosomus_neogaeus,...,cottus_cognatus,catostomus_commersonii,chrosomus_eos,hiodon_alosoides,perca_flavescens,cottus_ricei,culaea_inconstans,thymallus_arcticus,pimephales_promelas,etheostoma_exile
260,Christina River,Christina-3,55.719685,-111.220095,couesius_plumbeus,2017,55.72_-111.22,NaN,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
123,Christina River,Christina-1,56.666930,-111.064791,chrosomus_eos,2017,56.667_-111.065,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
148,Christina River,Christina-1,56.666930,-111.064791,cottus_cognatus,2017,56.667_-111.065,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
203,Christina River,Christina-3,55.719685,-111.220095,esox_lucius,2017,55.72_-111.22,NaN,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
130,Christina River,Christina-1,56.666930,-111.064791,chrosomus_neogaeus,2017,56.667_-111.065,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [94]:
print(len(df_2017))
df_2017.drop(columns=['species'], inplace=True)
df_2017.drop_duplicates(inplace=True)
print(len(df_2017))
display(df_2017)

60
11


,waterbody_name,site_description,latitude,longitude,year,lat_long,rhinichthys_cataractae,catostomus_catostomus,chrosomus_neogaeus,esox_lucius,...,cottus_cognatus,catostomus_commersonii,chrosomus_eos,hiodon_alosoides,perca_flavescens,cottus_ricei,culaea_inconstans,thymallus_arcticus,pimephales_promelas,etheostoma_exile
0,Christina River,Christina-2,56.753900,-110.508300,2017,56.754_-110.508,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Jackfish River,Jackfish,55.672346,-111.099532,2017,55.672_-111.1,1.0,1.0,NaN,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Sunday Creek,Sunday-1,55.584120,-110.893442,2017,55.584_-110.893,NaN,1.0,NaN,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,Christina River,Christina-1,56.666930,-111.064791,2017,56.667_-111.065,1.0,1.0,1.0,1.0,...,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
166,Christina River,Christina-4,55.888699,-111.539965,2017,55.889_-111.54,NaN,1.0,1.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
199,Christina River,Christina-3,55.719685,-111.220095,2017,55.72_-111.22,NaN,1.0,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
266,Unnamed Creek,Unnamed,55.618160,-110.718805,2017,55.618_-110.719,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,Unnamed Creek,Unnamed,55.584273,-110.823274,2017,55.584_-110.823,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,Sawbones Creek,Sawbones,55.650398,-110.817812,2017,55.65_-110.818,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,Sunday Creek,Sunday-2,55.560995,-111.090609,2017,55.561_-111.091,NaN,NaN,NaN,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [95]:
# 2019

df_2019 = pd.read_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/To_look_at/WildFishCommFishHealth-CommDePoissonsSauvagesSanteDesPoissons-2019-1.csv')
df_2019.drop(columns=['Fork Length (mm) / La longueur à la fourche (mm)', 'Total Weight (g) / Poids total (g)', 
                      'Notes / Remarques', 'Site Name / Nom du site', 'Reach Code / Code de allonge'], inplace=True)

df_2019.head()

,Waterbody Name / Le nom du plan d'eau,Site & Description / Site et description,Latitude (Decimal Degrees) / Latitude (degrés décimaux),Longitude (Decimal Degrees) / Longitude (degrés décimaux),Date (yyyy-mm-dd) / Date (aaaa-mm-jj),Species / Espèce
0,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019-09-24,SLIMY SCULPIN -SLSC
1,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019-09-24,SLIMY SCULPIN -SLSC
2,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019-09-24,SLIMY SCULPIN -SLSC
3,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019-09-24,SLIMY SCULPIN -SLSC
4,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019-09-24,SLIMY SCULPIN -SLSC


In [96]:
df_2019.rename(columns=wild_fish_com_col_names, inplace=True)
df_2019['year'] = pd.DatetimeIndex(pd.to_datetime(df_2019['date'])).year

print(len(df_2019))
df_2019.drop(columns=['date'], inplace=True)
df_2019.drop_duplicates(inplace=True)
print(len(df_2019))

df_2019['lat_long'] = df_2019['latitude'].round(3).astype(str) + '_' +  df_2019['longitude'].round(3).astype(str)

df_2019['species'] = df_2019['species'].str.split('-').str.get(0).str.replace(' ', '_').str.rstrip('_').str.lower()
df_2019.replace(dict_species_name, inplace=True)
display(df_2019.sample(5))

2175
82


,waterbody_name,site_description,latitude,longitude,species,year,lat_long
1194,Dunkirk River,150m upstream of start co-ordinate (56.859746;...,56.859693,-112.711679,rhinichthys_cataractae,2019,56.86_-112.712
1532,Muskeg River,150m upstream of start co-ordinate (57.130550;...,57.130494,-111.593965,perca_flavescens,2019,57.13_-111.594
944,Firebag River,150m upstream of start co-ordinate (57.516291;...,57.515820,-111.113120,couesius_plumbeus,2019,57.516_-111.113
1536,Muskeg River,150m upstream of start co-ordinate (57.130550;...,57.130494,-111.593965,couesius_plumbeus,2019,57.13_-111.594
645,Steepbank River,150m upstream of start co-ordinate (57.024114;...,57.023653,-111.473053,catostomus_commersonii,2019,57.024_-111.473


In [97]:
for col in df_2019['species'].unique():
    df_2019[col] = np.where(df_2019.isin([col]).any(1), 1, np.nan)
    df_2019[col] = df_2019[col].fillna(df_2019.groupby('lat_long')[col].transform('mean'))

display(df_2019.sample(5))

,waterbody_name,site_description,latitude,longitude,species,year,lat_long,cottus_cognatus,catostomus_catostomus,margariscus_margarita,...,cottus_ricei,cyprinidae_spp,cottoidea_spp,pungitius_pungitius,thymallus_arcticus,chrosomus_eos,esox_lucius,culaea_inconstans,perca_flavescens,chrosomus_neogaeus
355,Steepbank River,150m upstream of start co-ordinate (56.991100;...,56.991623,-111.333979,cottus_cognatus,2019,56.992_-111.334,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1864,Calumet River,150m upstream of start co-ordinate (57.403308...,57.403831,-111.684532,chrosomus_eos,2019,57.404_-111.685,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0
1525,Muskeg River,150m upstream of start co-ordinate (57.130550;...,57.130494,-111.593965,rhinichthys_cataractae,2019,57.13_-111.594,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
636,Steepbank River,150m upstream of start co-ordinate (57.024114;...,57.023653,-111.473053,couesius_plumbeus,2019,57.024_-111.473,1.0,NaN,NaN,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687,Firebag River,150m upstream of start co-ordinate (57.335257;...,57.335136,-110.476200,rhinichthys_cataractae,2019,57.335_-110.476,NaN,1.0,NaN,...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
print(len(df_2019))
df_2019.drop(columns=['species'], inplace=True)
df_2019.drop_duplicates(inplace=True)
print(len(df_2019))
display(df_2019)

82
13


,waterbody_name,site_description,latitude,longitude,year,lat_long,cottus_cognatus,catostomus_catostomus,margariscus_margarita,couesius_plumbeus,...,cottus_ricei,cyprinidae_spp,cottoidea_spp,pungitius_pungitius,thymallus_arcticus,chrosomus_eos,esox_lucius,culaea_inconstans,perca_flavescens,chrosomus_neogaeus
0,Steepbank River,Upper RAMP - 150m upstream of start co-ordinat...,56.822919,-110.980353,2019,56.823_-110.98,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,Steepbank River,Upper ECCC - 150m upstream of start co-ordinat...,56.863433,-111.126440,2019,56.863_-111.126,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,Steepbank River,150m upstream of start co-ordinate (56.991100;...,56.991623,-111.333979,2019,56.992_-111.334,1.0,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631,Steepbank River,150m upstream of start co-ordinate (57.024114;...,57.023653,-111.473053,2019,57.024_-111.473,1.0,NaN,NaN,1.0,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,Firebag River,150m upstream of start co-ordinate (57.335257;...,57.335136,-110.476200,2019,57.335_-110.476,NaN,1.0,NaN,1.0,...,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
830,Firebag River,150m upstream of start co-ordinate (57.436916;...,57.434560,-110.892410,2019,57.435_-110.892,NaN,1.0,NaN,1.0,...,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
943,Firebag River,150m upstream of start co-ordinate (57.516291;...,57.515820,-111.113120,2019,57.516_-111.113,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
1013,High Hills River,150m upstream of start co-ordinate (56.753725;...,56.753541,-110.509801,2019,56.754_-110.51,1.0,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,Dunkirk River,150m upstream of start co-ordinate (56.859746;...,56.859693,-112.711679,2019,56.86_-112.712,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1252,Horse River,150m upstream of start co-ordinate (56.361770;...,56.361678,-112.175221,2019,56.362_-112.175,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [99]:
# merge 2013, 2017, 2019

wild_fish_comm_oil_sands = pd.concat([df_2013, df_2017, df_2019], sort=False)
print(len(wild_fish_comm_oil_sands))
display(wild_fish_comm_oil_sands.sample(10))

wild_fish_comm_oil_sands.to_csv('/mnt/c/Users/imrit/Downloads/Sharma_fish_project/Alberta/Processed/wild_fish_comm_oil_sands_occurence_2013_2017_2019.csv', 
                                index=False)

34


,waterbody_name,site_description,latitude,longitude,year,lat_long,chrosomus_neogaeus,rhinichthys_cataractae,catostomus_catostomus,margariscus_margarita,...,hiodon_alosoides,perca_flavescens,culaea_inconstans,thymallus_arcticus,pimephales_promelas,etheostoma_exile,salmonidae_spp,cyprinidae_spp,cottoidea_spp,pungitius_pungitius
122,Christina River,Christina-1,56.666930,-111.064791,2017,56.667_-111.065,1.0,1.0,1.0,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,Unnamed Creek,Unnamed,55.584273,-110.823274,2017,55.584_-110.823,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,Ells River,Middle RAMP,57.221307,-111.988604,2013,57.221_-111.989,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,Steepbank River,Upper ECCC - 150m upstream of start co-ordinat...,56.863433,-111.126440,2019,56.863_-111.126,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
277,Sunday Creek,Sunday-2,55.560995,-111.090609,2017,55.561_-111.091,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN
51,Ells River,Lower Ells,57.266300,-111.734200,2013,57.266_-111.734,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Tar River,150m upstream of start co-ordinate (57.393686;...,57.394026,-111.985955,2019,57.394_-111.986,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
166,Christina River,Christina-4,55.888699,-111.539965,2017,55.889_-111.54,1.0,NaN,1.0,1.0,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
20,High Hills River,High Hills - East of Fort McMurray,56.753900,-110.508300,2013,56.754_-110.508,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1252,Horse River,150m upstream of start co-ordinate (56.361770;...,56.361678,-112.175221,2019,56.362_-112.175,NaN,1.0,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN


### That's it Folks